# My Merrill Edge Portfolio performance analysis 2022 to April 2025

* The result is terrible. We lost over 2M.
* This is because we didn't liquidate ABNB on IPO (2020-12-10). In the 53 months since then, ABNB lagged SP500 by a lot.
  - SP500 growed by 52%
  - ABNB shrank by 17%
  - This single mistake resulted in our big loss


In [4]:

from fmp_data import FMPPriceLoader

pl = FMPPriceLoader()




In [50]:
pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Closed.csv")

,Symbol,Acquisition Date,Liquidation Date,Quantity,Acquisition Price ($),Acquisition Cost ($),Liquidation Price ($),Liquidation Amount ($),Gain/Loss ($)
0,TUSK,04/22/2019 (Long Term),06/30/2021,85,$17.95,"$1,525.75",$4.40,$373.99,"-$1,151.76"
1,TUSK,06/06/2019 (Long Term),06/30/2021,100,$6.03,$603.00,$4.40,$439.99,-$163.01
2,TUSK,06/06/2019 (Long Term),06/30/2021,300,$6.09,"$1,827.00",$4.40,"$1,319.99",-$507.01
3,TUSK,01/08/2020 (Long Term),06/30/2021,"1,000",$2.23,"$2,228.20",$4.40,"$4,400.00","$2,171.80"
4,BABA,12/10/2020 (Short Term),08/13/2021,22,$264.79,"$5,825.27",$187.68,"$4,128.99","-$1,696.28"
...,...,...,...,...,...,...,...,...,...
68,ABNB,12/09/2020 (Long Term),02/18/2025,439,144.71,"63,527.69",145,"65,134.16","1,606.47"
69,ABNB,12/09/2020 (Long Term),02/18/2025,561,144.71,"81,182.31",145,"83,235.22","2,052.91"
70,CCL,11/18/2024 (Short Term),04/08/2025,590,$24.54,"$14,475.65",$16.95,"$10,003.17","-$4,472.48"
71,GM,11/18/2024 (Short Term),04/08/2025,225,$56.67,"$12,750.73",$43.00,"$9,674.73","-$3,076.00"


In [48]:
pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Holdings.csv")

,Symbol,Acquisition Date,Quantity,Unit Cost ($),Cost Basis ($),Value($),Gain/Loss ($),Gain/Loss (%)
0,NVDA,04/04/2025 (Short Term),"1,000",103.84,"103,836.53","109,560.00","5,723.47",5.51
1,NVDA,11/29/2024 (Short Term),808,136.7,"110,451.10","88,524.48","-21,926.62",-19.85
2,NVDA,10/22/2024 (Short Term),196,143.15,"28,057.38","21,473.76","-6,583.62",-23.46
3,NVDA,08/21/2024 (Short Term),"4,330",128,"554,240.00","474,394.80","-79,845.20",-14.41
4,NVDA,04/17/2023 (Long Term),830,26.59,"22,073.02","90,934.80","68,861.78",311.97
...,...,...,...,...,...,...,...,...
93,BAC,07/08/2019 (Long Term),99.761,29.28,"2,920.68","3,985.46","1,064.78",36.46
94,JPM,08/14/2019 (Long Term),50,106.66,"5,333.00","12,242.00","6,909.00",129.55
95,JPM,10/31/2019 (Long Term),0.215,125.58,27,52.64,25.64,94.96
96,JPM,07/16/2019 (Long Term),29.785,114.27,"3,403.43","7,292.56","3,889.13",114.27


In [9]:

from fmp_fetch.fmp_online import FMPOnline
from functools import cache

fmp_online = FMPOnline()

@cache
def fmp_price(symbol, date):
    if symbol == 'TWTR':
        return 53.70
    if symbol == 'BRK.B':
        symbol = 'BRK-B'
    return fmp_online.get_price(symbol, date)['adjClose']

@cache
def fmp_pe(symbol, date):
    return fmp_online.get_pe_ratio(symbol, date)

@cache
def fmp_price_to_fcf(symbol, date):
    return fmp_online.get_price_to_fcf(symbol, date)

@cache
def spx_price(date: str) -> float:
    return fmp_online.get_index_price('^GSPC', date)




# Counterfactual Analysis for closed positions: what if instead of buying the stock, we buy the S&P 500 and sell on the liquidation date?

In [54]:
TODAY = '2025-04-29'

closed = pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Closed.csv")

closed['Acquisition Date'] = closed['Acquisition Date'].apply(lambda x: x.replace(' (Long Term)', '').replace(' (Short Term)', ''))
closed['Acquisition Date'] = pd.to_datetime(closed['Acquisition Date']).dt.strftime('%Y-%m-%d')
closed['Liquidation Date'] = pd.to_datetime(closed['Liquidation Date']).dt.strftime('%Y-%m-%d')
closed['Acquisition Cost ($)'] = closed['Acquisition Cost ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Liquidation Amount ($)'] = closed['Liquidation Amount ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Liquidation Price ($)'] = closed['Liquidation Price ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
closed['Quantity'] = closed['Quantity'].apply(lambda x: x.replace(',', '')).astype(float)

closed['acquisition_pe'] = closed.apply(lambda x: fmp_pe(x['Symbol'], x['Acquisition Date']), axis=1)
closed['liquidation_pe'] = closed.apply(lambda x: fmp_pe(x['Symbol'], x['Liquidation Date']), axis=1)
closed['acquisition_price_to_fcf'] = closed.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Acquisition Date']), axis=1)
closed['liquidation_price_to_fcf'] = closed.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Liquidation Date']), axis=1)
closed['gain'] = closed['Liquidation Amount ($)'] / closed['Acquisition Cost ($)'] - 1
closed['spx_acquisition_price'] = closed['Acquisition Date'].apply(lambda x: spx_price(x))
closed['spx_liquidation_price'] = closed['Liquidation Date'].apply(lambda x: spx_price(x))
closed['spx_liquidation_counterfactual'] = closed['Acquisition Cost ($)'] / closed['spx_acquisition_price'] * closed['spx_liquidation_price']
closed['minus_spx'] = closed['Liquidation Amount ($)'] - closed['spx_liquidation_counterfactual']
closed['minus_spx_percent'] = closed['minus_spx'] / closed['Acquisition Cost ($)']
closed['spx_gain'] = closed['spx_liquidation_price'] / closed['spx_acquisition_price'] - 1

closed['today'] = TODAY
closed['today_price'] = closed['Symbol'].apply(lambda x: fmp_price(x, TODAY))
closed['today_liquidation_amount'] = closed['Quantity'] * closed['today_price']
closed['today_spx_amount'] = closed['Acquisition Cost ($)'] / closed['spx_acquisition_price'] * spx_price(TODAY)
closed['today_minus_spx'] = closed['today_liquidation_amount'] - closed['today_spx_amount']
closed['sell_early_win'] = closed['minus_spx'] - closed['today_minus_spx'] # Compared to not selling, how much gain over S&P500 would be today?
closed['sell_early_win_percent'] = closed['sell_early_win'] / closed['Acquisition Cost ($)']

In [55]:
print(f"Compared to S&P 500, our gain is {sum(closed['minus_spx']):,.2f} USD"
    + f", or {sum(closed['minus_spx']) / sum(closed['Acquisition Cost ($)']) * 100:.2f}%\n"
    + f"If holding to {TODAY}, the gain would have been {sum(closed['today_minus_spx']):,.2f} USD")

Compared to S&P 500, our gain is -1,262,635.75 USD, or -51.94%
If holding to 2025-04-29, the gain would have been -1,026,249.90 USD


In [56]:
abnb_closed = closed[closed['Symbol'] == 'ABNB']
non_abnb_closed = closed[closed['Symbol'] != 'ABNB']
print(f"abnb_closed: {abnb_closed.minus_spx.sum():,.2f}, if hold until today: {abnb_closed.today_minus_spx.sum():,.2f}," +
     f" selling gain %: {abnb_closed.sell_early_win.sum() / abnb_closed['Acquisition Cost ($)'].sum() * 100:,.2f}%")
print(f"non_abnb_closed: {non_abnb_closed.minus_spx.sum():,.2f}, {non_abnb_closed.today_minus_spx.sum():,.2f}," +
     f" selling gain %: {non_abnb_closed.sell_early_win.sum() / non_abnb_closed['Acquisition Cost ($)'].sum() * 100:,.2f}%")


abnb_closed: -1,024,085.67, if hold until today: -983,722.88, selling gain %: -2.65%
non_abnb_closed: -238,550.08, -42,527.02, selling gain %: -21.53%


# Good deals & bad deals from closed positions

In [66]:
columns = ['Symbol', 'Acquisition Date', 'acquisition_pe', 'acquisition_price_to_fcf', 'Acquisition Cost ($)', 'gain', 'spx_gain', 'minus_spx_percent']
closed.sort_values(by='minus_spx_percent', ascending=True)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,acquisition_price_to_fcf,Acquisition Cost ($),gain,spx_gain,minus_spx_percent
0,TUSK,2019-04-22,3.410095,4.131386,1525.75,-0.754881,0.477835,-1.232716
10,T,2019-08-20,6.835151,5.035256,1758.75,-0.304341,0.621970,-0.926311
11,T,2019-08-20,6.835151,5.035256,3516.00,-0.304042,0.621970,-0.926011
57,UPS,2019-09-04,18.678976,18.261873,28.80,0.118056,1.042815,-0.924759
12,T,2019-09-16,7.290082,5.370391,3724.56,-0.343012,0.569247,-0.912259
41,BIDU,2019-05-21,12.198700,45.248061,3088.80,-0.119318,0.769949,-0.889267
58,UPS,2019-12-04,17.573859,24.422383,29.03,0.132277,0.927980,-0.795703
2,TUSK,2019-06-06,1.158825,1.403936,1827.00,-0.277510,0.511347,-0.788857
1,TUSK,2019-06-06,1.158825,1.403936,603.00,-0.270332,0.511347,-0.781679
59,UPS,2019-07-16,16.373070,16.007458,3139.66,0.245358,0.997756,-0.752399


In [67]:
closed.sort_values(by='minus_spx_percent', ascending=False)[columns].head(20)

,Symbol,Acquisition Date,acquisition_pe,acquisition_price_to_fcf,Acquisition Cost ($),gain,spx_gain,minus_spx_percent
50,GOOG,2018-11-26,44.435912,34.028068,1726.26,2.530007,1.174591,1.355416
51,GOOG,2018-12-26,44.044257,33.728146,3811.74,2.702349,1.355902,1.346446
53,GOOG,2019-01-25,40.237109,32.976823,4444.87,2.365585,1.181682,1.183903
52,GOOG,2019-02-25,40.911432,33.529473,4595.82,2.255040,1.079196,1.175845
49,GOOG,2019-03-25,43.994054,36.055874,1363.90,1.998182,1.077524,0.920658
38,EDU,2021-07-26,7.965036,5.819127,2.42,0.611570,-0.061276,0.672846
39,EDU,2021-07-26,7.965036,5.819127,191.57,0.610116,-0.061276,0.671392
3,TUSK,2020-01-08,1.925790,-1.478109,2228.20,0.974688,0.321068,0.653620
27,TWTR,2022-04-26,-179.061614,-105.188062,59188.24,0.065901,-0.072624,0.138525
26,TWTR,2022-04-26,-179.061614,-105.188062,37106.10,0.064833,-0.072624,0.137457


# Sell V.S Hold

In [84]:
columns = ['Symbol', 'Acquisition Date', 'Liquidation Date', 'acquisition_pe','Acquisition Price ($)', 'Liquidation Price ($)', 'today_price', 'liquidation_price_to_fcf', 'minus_spx_percent', 'sell_early_win', 'sell_early_win_percent']
non_abnb_closed.groupby('Symbol').agg({'sell_early_win_percent': 'min'}).reset_index().merge(
    non_abnb_closed, on=['Symbol', 'sell_early_win_percent']
).sort_values(by='sell_early_win_percent', ascending=True)[columns].head(20)

,Symbol,Acquisition Date,Liquidation Date,acquisition_pe,Acquisition Price ($),Liquidation Price ($),today_price,liquidation_price_to_fcf,minus_spx_percent,sell_early_win,sell_early_win_percent
6,CRS,2020-04-15,2021-11-18,4.621050,$19.31,30.69,197.44,9.645697,-0.100905,-35860.537289,-8.327827
14,META,2018-10-16,2023-02-17,24.411895,$156.16,170.42,554.44,18.018428,-0.360365,-11765.308172,-1.931830
8,EDU,2021-08-12,2022-04-26,8.255277,$20.40,9.87,48.88,-1.683357,-0.452261,-112018.347676,-1.601868
3,BRK-B,2019-08-23,2021-11-18,18.117502,198.11,279.63,534.57,22.216302,-0.240918,-10745.008386,-0.986147
1,BABA,2021-11-18,2024-11-11,20.298531,$142.55,95.46,118.88,14.548711,-0.606007,-3015.193775,-0.257949
17,TSLA,2021-10-19,2024-10-08,392.249590,$292.33,244.35,292.03,452.678790,-0.436643,-6479.655608,-0.205234
22,WFC,2019-08-14,2021-11-18,7.737524,$44.33,49.65,71.10,-4.417719,-0.536050,-808.882320,-0.182489
5,CLS,2024-11-15,2024-11-18,26.759627,$81.12,86.00,87.05,33.498616,0.056215,-5503.935582,-0.069661
21,VOO,2020-08-18,2025-01-21,1.000000,312.5,537.50,509.49,1.000000,-0.064549,-0.027226,-0.054451
13,LC,2025-01-17,2025-04-08,35.740987,$16.34,8.90,11.01,-0.356326,-0.286151,-963.919359,-0.032780


In [86]:
non_abnb_closed.groupby('Symbol').agg({'sell_early_win_percent': 'min'}).reset_index().merge(
    non_abnb_closed, on=['Symbol', 'sell_early_win_percent']
).sort_values(by='sell_early_win_percent', ascending=False)[columns].head(20)

,Symbol,Acquisition Date,Liquidation Date,acquisition_pe,Acquisition Price ($),Liquidation Price ($),today_price,liquidation_price_to_fcf,minus_spx_percent,sell_early_win,sell_early_win_percent
0,AMZN,2021-01-12,2021-11-18,89.673529,"3,126.81",3603.89,187.39,261.051869,-0.085077,41208.479138,1.317908
18,TUSK,2019-04-22,2021-06-30,3.410095,$17.95,4.40,2.61,15.959208,-1.232716,814.977130,0.534149
12,KHC,2019-02-22,2021-11-18,3.055473,$34.63,35.40,28.87,9.310657,-0.662516,2502.866177,0.495032
19,TWTR,2022-04-26,2022-10-31,-179.061614,$50.90,0.00,53.70,-50.140721,0.137457,15373.665052,0.414316
10,GOLD,2020-08-28,2021-11-18,10.696291,$29.27,20.73,18.90,10.570787,-0.632959,3141.489523,0.306605
2,BIDU,2019-05-21,2024-02-28,12.198700,$118.80,104.63,89.46,11.866687,-0.889267,923.816749,0.299086
7,DJT,2024-11-04,2024-11-06,-8.611913,$32.22,35.19,24.98,-124.281486,0.054337,2503.764681,0.252339
11,GOOG,2019-03-25,2024-10-30,43.994054,$60.27,180.72,162.06,36.050428,0.920658,298.893971,0.219147
16,T,2019-09-16,2021-11-18,7.290082,$37.25,24.47,27.30,3.701460,-0.912259,780.802138,0.209636
15,PFE,2021-12-15,2024-02-28,21.323488,$57.01,27.03,23.79,17.480595,-0.602341,1615.350282,0.161006


In [88]:
fmp_price('AMZN', '2021-01-12'), fmp_price('AMZN', '2021-1-18')

(156.04, 188.47)

In [21]:
print(f"Besides ABNB, our gain against SPX counterfactur is {non_abnb_closed.minus_spx.sum():,.2f} USD" +
    f"or {non_abnb_closed.minus_spx.sum() / non_abnb_closed['Acquisition Cost ($)'].sum() * 100:.2f}%")

Besides ABNB, our gain against SPX counterfactur is -218,068.93 USDor -21.39%


# Counterfactual Analysis for current holdings (2025-04-29): what if we bought S&P 500 instead of the stock?

In [57]:
from datetime import timedelta
current = pd.read_csv("data/Stock Investment Performance Until 2025-04-29 - Holdings.csv")

current['Acquisition Date'] = current['Acquisition Date'].apply(lambda x: x.replace(' (Long Term)', '').replace(' (Short Term)', ''))
current['Acquisition Date'] = pd.to_datetime(current['Acquisition Date']).dt.strftime('%Y-%m-%d')
current['Liquidation Date'] = (pd.to_datetime(TODAY) - timedelta(days=1)).strftime('%Y-%m-%d')
current['Cost Basis ($)'] = current['Cost Basis ($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
current['Value($)'] = current['Value($)'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
current['Quantity'] = current['Quantity'].apply(lambda x: x.replace(',', '')).astype(float)


current['acquisition_pe'] = current.apply(lambda x: fmp_pe(x['Symbol'], x['Acquisition Date']), axis=1)
current['liquidation_pe'] = current.apply(lambda x: fmp_pe(x['Symbol'], x['Liquidation Date']), axis=1)
current['acquisition_price_to_fcf'] = current.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Acquisition Date']), axis=1)
current['liquidation_price_to_fcf'] = current.apply(lambda x: fmp_price_to_fcf(x['Symbol'], x['Liquidation Date']), axis=1)
current['gain'] = current['Value($)'] / current['Cost Basis ($)'] - 1   
current['spx_acquisition_price'] = current['Acquisition Date'].apply(lambda x: spx_price(x))
current['spx_liquidation_price'] = current['Liquidation Date'].apply(lambda x: spx_price(x))
current['spx_liquidation_counterfactual'] = current['Cost Basis ($)'] / current['spx_acquisition_price'] * current['spx_liquidation_price']
current['minus_spx'] = current['Value($)'] - current['spx_liquidation_counterfactual']
current['minus_spx_percent'] = current['minus_spx'] / current['Cost Basis ($)']
current['spx_gain'] = current['spx_liquidation_price'] / current['spx_acquisition_price'] - 1

current.head()

,Symbol,Acquisition Date,Quantity,Unit Cost ($),Cost Basis ($),Value($),Gain/Loss ($),Gain/Loss (%),Liquidation Date,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,gain,spx_acquisition_price,spx_liquidation_price,spx_liquidation_counterfactual,minus_spx,minus_spx_percent,spx_gain
0,NVDA,2025-04-04,1000.0,103.84,103836.53,109560.00,"5,723.47",5.51,2025-04-28,36.767675,36.622644,41.015593,43.870543,0.055120,5074.09,5528.74,113140.519162,-3580.519162,-0.034482,0.089602
1,NVDA,2024-11-29,808.0,136.7,110451.10,88524.48,"-21,926.62",-19.85,2025-04-28,64.219057,36.622644,72.759321,43.870543,-0.198519,6032.39,5528.74,101229.432217,-12704.952217,-0.115028,-0.083491
2,NVDA,2024-10-22,196.0,143.15,28057.38,21473.76,"-6,583.62",-23.46,2025-04-28,83.116301,36.622644,90.006213,43.870543,-0.234648,5851.19,5528.74,26511.181333,-5037.421333,-0.179540,-0.055108
3,NVDA,2024-08-21,4330.0,128,554240.00,474394.80,"-79,845.20",-14.41,2025-04-28,74.374530,36.622644,80.539794,43.870543,-0.144062,5620.84,5528.74,545158.527480,-70763.727480,-0.127677,-0.016385
4,NVDA,2023-04-17,830.0,26.59,22073.02,90934.80,"68,861.78",311.97,2025-04-28,113.279493,36.622644,139.878457,43.870543,3.119726,4151.32,5528.74,29396.911969,61537.888031,2.787923,0.331803


In [58]:
print(f"Compared to S&P 500, our gain is {sum(current['minus_spx'])} USD")
print(f"Compared to S&P 500, our gain excluding ABNB is {sum(current[current.Symbol != 'ABNB']['minus_spx'])} USD")
print(f"Excluding ABNB, our gain over holding SPX from closing positions is {non_abnb_closed.minus_spx.sum():,.2f} USD")

Compared to S&P 500, our gain is 20019.246480895268 USD
Compared to S&P 500, our gain excluding ABNB is 249588.92264052184 USD
Excluding ABNB, our gain over holding SPX from closing positions is -238,550.08 USD


In [341]:
columns = ['Symbol', 'acquisition_pe', 'liquidation_pe', 'acquisition_price_to_fcf', 'liquidation_price_to_fcf', 'Cost Basis ($)', 'Value($)', 'gain', 'spx_acquisition_price', 'spx_liquidation_price', 'spx_gain','minus_spx', 'minus_spx_percent']
current.sort_values(by='minus_spx_percent', ascending=True)[columns].head(20)

,Symbol,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,Cost Basis ($),Value($),gain,spx_acquisition_price,spx_liquidation_price,spx_gain,minus_spx,minus_spx_percent
61,ABNB,-49.930282,29.386781,-33.720460,17.303511,360183.19,312618.40,-0.132057,3672.820068,5528.75,0.505315,-229570.646740,-0.637372
90,BAC,10.422741,11.516396,22.343183,-36.898183,36.22,40.55,0.119547,3240.020020,5528.75,0.706394,-21.255583,-0.586847
91,BAC,8.924067,11.516396,17.616178,-36.898183,36.00,48.90,0.358333,2961.790039,5528.75,0.866692,-18.300915,-0.508359
93,BAC,8.819569,11.516396,17.409899,-36.898183,2920.68,3985.46,0.364566,2975.949951,5528.75,0.857810,-1440.608924,-0.493244
87,BAC,9.119128,11.516396,18.001231,-36.898183,2964.00,3995.01,0.347844,3006.729980,5528.75,0.838792,-1455.168468,-0.490948
86,BAC,9.119128,11.516396,18.001231,-36.898183,118.52,159.80,0.348296,3006.729980,5528.75,0.838792,-58.133587,-0.490496
92,BAC,8.756871,11.516396,17.286132,-36.898183,2893.61,3995.01,0.380632,3004.040039,5528.75,0.840438,-1330.500339,-0.459806
88,BAC,7.980108,11.516396,15.752796,-36.898183,1326.25,1997.51,0.506134,2840.600098,5528.75,0.946332,-583.812409,-0.440198
89,BAC,7.980108,11.516396,15.752796,-36.898183,5304.80,7990.02,0.506187,2840.600098,5528.75,0.946332,-2334.880370,-0.440145
69,AMZN,89.863174,33.138057,63.146836,59.714979,98176.23,116274.80,0.184348,3795.540039,5528.75,0.456644,-26733.000215,-0.272296


In [342]:
current.sort_values(by='minus_spx_percent', ascending=False)[columns].head(20)

,Symbol,acquisition_pe,liquidation_pe,acquisition_price_to_fcf,liquidation_price_to_fcf,Cost Basis ($),Value($),gain,spx_acquisition_price,spx_liquidation_price,spx_gain,minus_spx,minus_spx_percent
4,NVDA,113.279493,36.622644,139.878457,43.870543,22073.02,90934.80,3.119726,4151.319824,5528.75,0.331805,61537.833615,2.787921
25,AAPL,14.356523,33.264024,13.722397,32.551484,5446.38,25217.45,3.630130,2840.229980,5528.75,0.946585,14615.606617,2.683545
22,AAPL,15.509186,33.264024,14.830133,32.551484,5811.86,25388.40,3.368378,2844.739990,5528.75,0.943499,14093.054525,2.424879
21,AAPL,16.183063,33.264024,15.474505,32.551484,20188.00,84628.00,3.191995,2918.649902,5528.75,0.894283,46386.207139,2.297712
24,AAPL,16.246448,33.264024,15.535114,32.551484,23.10,96.48,3.176623,2847.600098,5528.75,0.941547,51.630260,2.235076
20,AAPL,17.707627,33.264024,16.932318,32.551484,4379.20,16925.60,2.864998,2997.959961,5528.75,0.844171,8849.607553,2.020827
23,AAPL,21.504144,33.264024,20.597266,32.551484,23.19,74.47,2.211298,3096.629883,5528.75,0.785409,33.066372,1.425889
57,GOOG,24.460520,19.978822,32.928681,27.471360,4894.90,14223.16,1.905710,2917.379883,5528.75,0.895108,4946.795775,1.010602
56,GOOG,27.993034,19.978822,30.308126,27.471360,5282.85,15049.71,1.848786,3003.669922,5528.75,0.840665,5325.753075,1.008121
74,MSFT,28.027114,31.347822,29.089914,41.514907,4060.60,11801.10,1.906245,2840.600098,5528.75,0.946332,3897.825523,0.959914


# Counterfactual Analysis Summary

In [60]:
# ABNB Performance
abnb_closed = closed[closed['Symbol'] == 'ABNB']
abnb_current = current[current['Symbol'] == 'ABNB']

# What if we had been holding SPX during the same period (same acquisition and liquidation dates)?
abnb_to_spx = abnb_closed.minus_spx.sum()
# What if we didn't sell but held those positions until today? How much those positions would compared with holding SPX until today?
abnb_not_closed_to_spx = abnb_closed.sell_early_win.sum()
# For current holding, what would it have been if we have bought SPX instead?
abnb_current_to_spx = abnb_current.minus_spx.sum()

print(f"ABNB closed to SPX: {abnb_to_spx:,.2f}$, {abnb_to_spx / abnb_closed['Acquisition Cost ($)'].sum():.2%}")
print(f"If ABNB not closed, compared to SPX today would be: {abnb_not_closed_to_spx:,.2f}$, {abnb_not_closed_to_spx / abnb_closed['Acquisition Cost ($)'].sum():.2%}")
print(f"ABNB current position compared to SPX: {abnb_current_to_spx:,.2f}$, {abnb_current_to_spx / abnb_current['Cost Basis ($)'].sum():.2%}")

ABNB closed to SPX: -1,024,085.67$, -67.34%
If ABNB not closed, compared to SPX today would be: -40,362.79$, -2.65%
ABNB current position compared to SPX: -229,569.68$, -63.74%


In [59]:
abnb_closed.sort_values(by='Liquidation Amount ($)', ascending=False)

,Symbol,Acquisition Date,Liquidation Date,Quantity,Acquisition Price ($),Acquisition Cost ($),Liquidation Price ($),Liquidation Amount ($),Gain/Loss ($),acquisition_pe,...,minus_spx,minus_spx_percent,spx_gain,today,today_price,today_liquidation_amount,today_spx_amount,today_minus_spx,sell_early_win,sell_early_win_percent
47,ABNB,2020-12-09,2024-08-19,4651.0,144.71,673046.21,119.00,553453.61,"-119,592.60",-49.930282,...,-474259.339932,-0.704646,0.526957,2025-04-29,125.49,583653.99,1.019023e+06,-435369.219820,-38890.120112,-0.057782
55,ABNB,2020-12-09,2024-11-11,1160.0,144.71,167863.60,137.28,159238.75,"-8,624.85",-49.930282,...,-115048.184079,-0.685367,0.633987,2025-04-29,125.49,145568.40,2.541533e+05,-108584.883894,-6463.300185,-0.038503
65,ABNB,2020-12-09,2025-01-21,1000.0,144.71,144710.00,130.00,131589.85,"-13,120.15",-49.930282,...,-106751.281147,-0.737691,0.647026,2025-04-29,125.49,125490.00,2.190977e+05,-93607.658529,-13143.622618,-0.090827
62,ABNB,2020-12-09,2024-11-27,870.0,144.71,125897.70,137.97,120034.91,"-5,862.79",-49.930282,...,-85590.824428,-0.679844,0.633276,2025-04-29,125.49,109176.30,1.906150e+05,-81438.662921,-4152.161507,-0.032980
69,ABNB,2020-12-09,2025-02-18,561.0,144.71,81182.31,145.00,83235.22,"2,052.91",-49.930282,...,-52249.952676,-0.643613,0.668900,2025-04-29,125.49,70399.89,1.229138e+05,-52513.896435,263.943759,0.003251
64,ABNB,2020-12-09,2024-12-09,500.0,144.71,72355.00,136.00,68974.83,"-3,380.17",-49.930282,...,-50266.853012,-0.694725,0.648009,2025-04-29,125.49,62745.00,1.095488e+05,-46803.829265,-3463.023747,-0.047862
68,ABNB,2020-12-09,2025-02-18,439.0,144.71,63527.69,145.00,65134.16,"1,606.47",-49.930282,...,-40887.213983,-0.643612,0.668900,2025-04-29,125.49,55090.11,9.618387e+04,-41093.762094,206.548111,0.003251
40,ABNB,2020-12-09,2023-06-12,413.0,144.71,59765.23,121.14,50030.32,"-9,734.91",-49.930282,...,-20574.051955,-0.344248,0.181362,2025-04-29,125.49,51827.37,9.048733e+04,-38659.962973,18085.911018,0.302616
63,ABNB,2020-12-09,2024-12-02,300.0,144.71,43413.00,137.31,41192.83,"-2,220.17",-49.930282,...,-30284.805392,-0.697598,0.646457,2025-04-29,125.49,37647.00,6.572930e+04,-28082.297559,-2202.507833,-0.050734
60,ABNB,2020-12-09,2024-11-18,301.0,144.71,43557.71,132.65,39926.54,"-3,631.17",-49.930282,...,-29968.569271,-0.688020,0.604655,2025-04-29,125.49,37772.49,6.594840e+04,-28175.905217,-1792.664054,-0.041156


In [61]:
# Non-ABNB Performance

non_abnb_closed = closed[closed['Symbol'] != 'ABNB']
non_abnb_current = current[current['Symbol'] != 'ABNB']

# What if we had been holding SPX during the same period (same acquisition and liquidation dates)?
non_abnb_to_spx = non_abnb_closed.minus_spx.sum()
# What if we didn't sell but held those positions until today? How much those positions would compared with holding SPX until today?
non_abnb_not_closed_to_spx = non_abnb_closed.sell_early_win.sum()
# For current holding, what would it have been if we have bought SPX instead?
non_abnb_current_to_spx = non_abnb_current.minus_spx.sum()

print(f"Non-ABNB closed to SPX: {non_abnb_to_spx:,.2f}$, {non_abnb_to_spx / non_abnb_closed['Acquisition Cost ($)'].sum():.2%}")
print(f"If non-ABNB not closed, compared to SPX today would be: {non_abnb_not_closed_to_spx:,.2f}$, {non_abnb_not_closed_to_spx / non_abnb_closed['Acquisition Cost ($)'].sum():.2%}")
print(f"Non-ABNB current position compared to SPX: {non_abnb_current_to_spx:,.2f}$, {non_abnb_current_to_spx / non_abnb_current['Cost Basis ($)'].sum():.2%}")

Non-ABNB closed to SPX: -238,550.08$, -26.21%
If non-ABNB not closed, compared to SPX today would be: -196,023.07$, -21.53%
Non-ABNB current position compared to SPX: 249,588.92$, 9.92%


In [65]:
# Net non-ABNB performance
non_abnb_net_to_spx = non_abnb_to_spx + non_abnb_current_to_spx
non_abnb_net_to_spx_not_closed = non_abnb_not_closed_to_spx + non_abnb_current_to_spx
non_abnb_base = non_abnb_closed['Acquisition Cost ($)'].sum() + non_abnb_current['Cost Basis ($)'].sum()
non_abnb_base_not_closed = non_abnb_closed['Acquisition Cost ($)'].sum() + non_abnb_current['Cost Basis ($)'].sum()
print(f"Excluding ABNB, the net performance against SPX (we have been holding and selling SPX instead of each stocks): {non_abnb_net_to_spx:,.2f}$, {non_abnb_net_to_spx / non_abnb_base:.2%}")
print(f"Excluding ABNB, if we never closed those positions, the net performance against SPX (we have been holding and selling SPX instead of each stocks): {non_abnb_net_to_spx_not_closed:,.2f}$, {non_abnb_net_to_spx_not_closed / non_abnb_base_not_closed:.2%}")


Excluding ABNB, the net performance against SPX (we have been holding and selling SPX instead of each stocks): 11,038.84$, 0.32%
Excluding ABNB, if we never closed those positions, the net performance against SPX (we have been holding and selling SPX instead of each stocks): 53,565.85$, 1.56%


# Summary

* Our performance exlucding ABNB roughly the same as S&P 500, being 0.32% ahead.
* ABNB's decision is terrible and resulted in the huge loss agaisnt S&P 500. Firstly, we should have sold it immediately at IPO. Secondly, we should not liquidated it a a low price of ~$120 in 2024. We then bought NVDA. While both performance badly since then, NVDA is even worth than ABNB.
  - This is only anecdotal but it's interesting to review that both NVDA and ABNB are expensive; ABNB being the less expensive one. Then both are laggard to the market with ABNB less behind.
* We did have some good stop loss sale, like BABA, GOLD, and TUSK. One can argue their performance is due to fundamental changes rather than emotional-driven panic.
* It's interesting to see how the following generalize as statistically accurate:
  - Depressive stocks – being lagged behind the market for more than a year – has much stronger safety margin.